# CaptainAgent
By: Jiale Liu, Linxin Song, Jieyu Zhang, Shaokun Zhang

In this notebook, we introduce CaptainAgent, an agent enhanced with the capability to call AutoBuild to break down and solve complex tasks. AutoBuild can initiate a group chat between a group of experts and converse to solve the task. The experts in nested chat can be retrieved from agent library. The agents can be equipped with tools for advanced coding.

````{=mdx}
:::info Requirements
Install `pyautogen`:
```bash
pip install pyautogen[autobuild]
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

## Setup API endpoint
In order to setup API, you should create a OAI_CONFIG_LIST file. The config list should look like the following:
```python
config_list = [
    {
        'model': 'gpt-4o-mini',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2024-08-01-preview',
    },
    {
        'model': 'gpt-3.5-turbo-16k',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2024-08-01-preview',
    },
]
```

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

In [1]:
import autogen

config_path = "OAI_CONFIG_LIST"
llm_config = {"temperature": 0}
config_list = autogen.config_list_from_json(
    config_path, filter_dict={"model": ["gpt-4o"]}
)  # You can modify the filter_dict to select your model

## Using CaptainAgent without libraries
We begin with demonstrating how to use CaptainAgent without retrieving from libaries. In this case, CaptainAgent will automatically generate a set of experts according to its identified subtask and initate the group chat.

In [ ]:
from autogen import UserProxyAgent
from autogen.agentchat.contrib.captainagent import CaptainAgent

llm_config = {"temperature": 0, "config_list": config_list}

## build agents
captain_agent = CaptainAgent(
    name="captain_agent",
    llm_config=llm_config,
    code_execution_config={"use_docker": False, "work_dir": "groupchat"},
    agent_config_save_path=None,  # If you'd like to save the created agents in nested chat for further use, specify the save directory here
)
captain_user_proxy = UserProxyAgent(name="captain_user_proxy", human_input_mode="NEVER")

In [3]:
result = captain_user_proxy.initiate_chat(
    captain_agent,
    message="Find a recent paper about large language models on arxiv and find its potential applications in software.",
    max_turns=1,
)

captain_user_proxy (to captain_agent):

Find a recent paper about large language models on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Expert_summoner (to CaptainAgent):

Find a recent paper about large language models on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
CaptainAgent (to Expert_summoner):

To find a recent paper about large language models on arXiv and explore its potential applications in software, I will follow these steps:

1. **Search for a Recent Paper**: I will search for a recent paper on arXiv related to large language models.
2. **Analyze the Paper**: I will analyze the

## Building Agents from library & Retrieve tools from tool library
One significant feature of CaptainAgent is that the agents and tools can be retrieved from a dedicated library. When CaptainAgent starts building experts for group chat, it will retrieve and select from agent library, then assign tools retreived from tool library to the experts.

For agent library, refer to `captainagent_expert_library.json` for samples. You can refer to [docs](https://ag2ai.github.io/ag2/docs/topics/captainagent/agent_library) on how to customize your own expert library.

For tool library, we provide a set of tools [here](https://github.com/ag2ai/ag2/tree/main/autogen/agentchat/contrib/captainagent/tools/README.md), the tools are categorized into three types: data analysis, information_retrieval, math.

### Using Agent Library Only
Below is an example that retrieves experts from library and build nested chat accordingly.


In [4]:
from autogen import UserProxyAgent
from autogen.agentchat.contrib.captainagent import CaptainAgent

nested_config = {
    # this is used to configure the autobuild building process
    "autobuild_build_config": {
        # this is used to configure the user proxy within nested chat
        "library_path_or_json": "captainagent_expert_library.json",
    }
}

Now let's instantiate the agents and ask it to solve a problem.

In [11]:
# build agents
captain_agent = CaptainAgent(
    name="captain_agent",
    llm_config=llm_config,
    nested_config=nested_config,
    code_execution_config={"use_docker": False, "work_dir": "groupchat"},
)
captain_user_proxy = UserProxyAgent(name="captain_user_proxy", human_input_mode="NEVER")

query = "find papers on LLM applications from arxiv in the last week, create a markdown table of different domains. After collecting the data, point out future research directions in light of the collected data."

result = captain_user_proxy.initiate_chat(captain_agent, message=query)

captain_user_proxy (to captain_agent):

find papers on LLM applications from arxiv in the last week, create a markdown table of different domains. After collecting the data, point out future research directions in light of the collected data.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Expert_summoner (to CaptainAgent):

find papers on LLM applications from arxiv in the last week, create a markdown table of different domains. After collecting the data, point out future research directions in light of the collected data.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
CaptainAgent (to Expert_summoner):

To address this task, I will follow these steps:

1. **Data Collection**: Gather recent papers on LL

## Using Both Agent Library and Tool Library
Now let's retrieve from both agent library and tool library while building experts for nested chat.

To run the following demo, it is required to install the dependencies from the tool library and obtain BING search api and Rapid API key for tools in library to fully function. Please follow the instructions [here](https://ag2ai.github.io/ag2/docs/topics/captainagent/tool_library).

In [12]:
from autogen import UserProxyAgent
from autogen.agentchat.contrib.captainagent import CaptainAgent

nested_config = {
    # this is used to configure the autobuild building process
    "autobuild_build_config": {
        "library_path_or_json": "captainagent_expert_library.json",
    },
    # this is used to configure the tool library
    "autobuild_tool_config": {
        "tool_root": "default",  # Setting the value to default will use the tool library we provide
    },
}

In [ ]:
# The function requires BING api key and Rapid API key to work. You can follow the instructions from docs to get one.
import os

os.environ["BING_API_key"] = ""  # set your bing api key here, if you donot need search engine, you can skip this step
os.environ["RAPID_API_KEY"] = (
    ""  # set your rapid api key here, in order for this example to work, you need to subscribe to the youtube transcription api(https://rapidapi.com/solid-api-solid-api-default/api/youtube-transcript3)
)

## build agents
captain_agent = CaptainAgent(
    name="captain_agent",
    llm_config=llm_config,
    nested_config=nested_config,
    code_execution_config={"use_docker": False, "work_dir": "groupchat"},
    agent_config_save_path=None,  # If you'd like to save the created agents in nested chat for further use, specify the save directory here
)
captain_user_proxy = UserProxyAgent(name="captain_user_proxy", human_input_mode="NEVER")

query = """# Task
Your task is to solve a question given by a user.

# Question
Examine the video at https://www.youtube.com/watch?v=1htKBjuUWec.

What does Teal'c say in response to the question "Isn't that hot?"
""".strip()
result = captain_user_proxy.initiate_chat(captain_agent, message=query)

captain_user_proxy (to captain_agent):

# Task
Your task is to solve a question given by a user.

# Question
Examine the video at https://www.youtube.com/watch?v=1htKBjuUWec.

What does Teal'c say in response to the question "Isn't that hot?"

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Expert_summoner (to CaptainAgent):

# Task
Your task is to solve a question given by a user.

# Question
Examine the video at https://www.youtube.com/watch?v=1htKBjuUWec.

What does Teal'c say in response to the question "Isn't that hot?"

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
CaptainAgent (to Expert_summoner):

To solve this task, I will follow these steps:

1. **Plan for solving the task**: 
   - Since the ta

/workspaces/ag2/autogen/agentchat/conversable_agent.py:1449: UserWarning: Extracted_response from ChatCompletion(id='chatcmpl-AW7akJMXAmetE5ACiGepg9aASnN0m', choices=[Choice(finish_reason='content_filter', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1732219042, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_d54531d9eb', usage=CompletionUsage(completion_tokens=5, prompt_tokens=486, total_tokens=491, completion_tokens_details=None, prompt_tokens_details=None), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filter

Computer_terminal (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: {'success': True, 'transcript': "Wow this coffee's great I was just thinking that yeah is that cinnamon chicory tea oak [Music] isn't that hot extremely"}
Wow this coffee's great I was just thinking that yeah is that cinnamon chicory tea oak [Music] isn't that hot extremely


--------------------------------------------------------------------------------

Next speaker: APIProcessing_Expert

APIProcessing_Expert (to chat_manager):

Based on the retrieved captions, Teal'c's response to the question "Isn't that hot?" is: 

"extremely"

This confirms the accuracy of the transcription regarding the specific dialogue. If there are any additional questions or tasks, feel free to let me know!

--------------------------------------------------------------------------------

Next speaker: APIProcessing_Expert

APIProcessing_Expert (to chat_manager):

TERMINATE

-------------------------------------------------

/workspaces/ag2/autogen/agentchat/conversable_agent.py:1449: UserWarning: Extracted_response from ChatCompletion(id='chatcmpl-AW7b4GKAuV0bxnAKxMg9drZu1f2Fw', choices=[Choice(finish_reason='content_filter', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1732219062, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_04751d0b65', usage=CompletionUsage(completion_tokens=5, prompt_tokens=766, total_tokens=771, completion_tokens_details=None, prompt_tokens_details=None), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filter

Creating agent VideoTranscription_Expert...
Adding user console proxy...
APIProcessing_Expert (to chat_manager):

You are given: (1) a task and advises from your manager with a specific plan and (2) a general task.
Collect information from the general task, follow the suggestions from manager to solve the task.

# General Task
# Task
Your task is to solve a question given by a user.

# Question
Examine the video at https://www.youtube.com/watch?v=1htKBjuUWec.

What does Teal'c say in response to the question "Isn't that hot?"

# Task and suggestions from manager
## Task description\nExamine the video at https://www.youtube.com/watch?v=1htKBjuUWec and find out what Teal'c says in response to the question "Isn't that hot?"\n\n## Plan for solving the task\n- Access the video and locate the moment when the question "Isn't that hot?" is asked.\n- Transcribe Teal'c's response accurately.\n\n## Output format\n- Provide the exact quote of Teal'c's response.\n\n## Constraints and conditions for

...

Computer_terminal (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: {'success': True, 'transcript': "Wow this coffee's great I was just thinking that yeah is that cinnamon chicory tea oak [Music] isn't that hot extremely"}
Wow this coffee's great I was just thinking that yeah is that cinnamon chicory tea oak [Music] isn't that hot extremely


--------------------------------------------------------------------------------


/workspaces/ag2/autogen/agentchat/conversable_agent.py:1449: UserWarning: Extracted_response from ChatCompletion(id='chatcmpl-AW7mCrD5g52QYPHIzvxHABFMbGko7', choices=[Choice(finish_reason='content_filter', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1732219752, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_d54531d9eb', usage=CompletionUsage(completion_tokens=5, prompt_tokens=789, total_tokens=794, completion_tokens_details=None, prompt_tokens_details=None), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filter


Next speaker: APIProcessing_Expert

APIProcessing_Expert (to chat_manager):

The captions from the video confirm that in response to the question "Isn't that hot?" Teal'c indeed replies, "extremely." This directly matches the task requirements, ensuring that the quote is accurate and faithful to the video content.

If there are no further questions or tasks related to this, I believe we can conclude the conversation. Do you agree, or is there anything else that needs attention?

--------------------------------------------------------------------------------


/workspaces/ag2/autogen/agentchat/conversable_agent.py:1449: UserWarning: Extracted_response from ChatCompletion(id='chatcmpl-AW7mItvGaj8eAzIEeogizVZWvXSIl', choices=[Choice(finish_reason='content_filter', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1732219758, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_04751d0b65', usage=CompletionUsage(completion_tokens=5, prompt_tokens=930, total_tokens=935, completion_tokens_details=None, prompt_tokens_details=None), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filter


Next speaker: APIProcessing_Expert

APIProcessing_Expert (to chat_manager):

TERMINATE

--------------------------------------------------------------------------------
Expert_summoner (to CaptainAgent):

Expert_summoner (to CaptainAgent):

***** Response from calling tool (call_ycX3M2UaldCUrHgxaIF6kYqq) *****
# Response from seek_agent_help: 
## Task
The task is to examine a YouTube video (https://www.youtube.com/watch?v=1htKBjuUWec) and find out what the character Teal'c says in response to the question "Isn't that hot?"

## Results
Teal'c's response to the question "Isn't that hot?" is "extremely," as confirmed by retrieving and reviewing the video captions using an API function.

## Reason for the results
The result was derived by accessing the video's captions through an API, which accurately reflected the dialogue in the video, confirming that the response "extremely" was indeed what Teal'c said in response to the question.

## Errors or issues in the conversation
The approach o

The ground truth answer to the question is 'Extremely', which the CaptainAgent answers correctly. Notably, with the assistance of tools, the agent can answer video-related questions. This shows the huge potential of what user customized tools can bring.

# Further Reading
For a full list of the configurables and the functionalities in CaptainAgent, please refer [here](https://ag2ai.github.io/ag2/docs/topics/captainagent/configurations).

For how to customize your own agent library, check [here](https://ag2ai.github.io/ag2/docs/topics/captainagent/agent_library).

For how to implement your own tool library that suits your need, check the documents [here](https://ag2ai.github.io/ag2/docs/topics/captainagent/tool_library).
